<h1>CS4618: Artificial Intelligence I</h1>
<h1>Error Estimation</h1>
<h2>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h2>

<h1>Initialization</h1>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from joblib import dump, load

<h1>Our Linear Model</h1>
<ul>
    <li>We'll repeat the code from the end of a previous lecture.</li>
</ul>

In [3]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("../datasets/dataset_corkA.csv")

In [4]:
# The features we want to select
features = ["flarea", "bdrms", "bthrms"]

# Extract those features and convert to a numpy array
X = df[features].values

# Extract the target values and convert to a numpy array
y = df["price"].values

In [5]:
# Fit the model

linear_model = LinearRegression()

linear_model.fit(X, y)

LinearRegression()

<h1>How Good Is This Model?</h1>
<ul>
    <li>We've built an estimator by learning a model from a dataset.</li>
    <li>We want to know how well it will do in practice, once we start to use it to make predictions.
        <ul>
            <li>This is called <b>error estimation</b>.</li>
        </ul>
    </li>
    <li>Easy right? 
        <ul>
            <li>The dataset comes with <em>actual</em> target values.</li>
            <li>We can ask the estimator to <em>predict</em> target values for each example in the dataset.</li>
            <li>So now we have actual and predicted values, we can compute the mean squared error.</li>
        </ul>
    </li>
</ul>

In [6]:
y_predictions = linear_model.predict(X)

In [7]:
mean_squared_error(y, y_predictions)

7648.571585887393

<ul>
    <li>But, for at least two reasons, we don't do this!
        <ul>
            <li>We might want to use a different performance measure than what we used as the loss function.</li>
            <li>We want to know how well the model <b>generalizes</b> to <b>unseen data</b>.</li>
        </ul>
    </li>
</ul>

<h1>Choosing a Different Performance Measure</h1>
<ul>
    <li>Often in machine learning, we use one measure during learning and another for evaluation.</li>
    <li>Our loss function (mean squared error or half of it!) was ideal for learning (why?)
        but may not be so good as a performance measure.
        <ul>
            <li>We could use <b>root mean squared error</b> (RMSE):
                $$\sqrt{\frac{1}{m}\sum_{i=1}^m(h_{\v{\beta}}(\v{x}^{(i)}) - \v{y}^{(i)})^2}$$
                i.e don't halve the MSE, and take its square root. RMSE is the standard deviation
                of the errors in the predictions.
            </li>
            <li>Or we could use <b>mean absolute error</b> (MAE):
                $$\frac{1}{m}\sum_{i=1}^m\abs(h_{\v{\beta}}(\v{x}^{i)}) - \v{y}^{(i)})$$
            </li>
        </ul>
    </li>
 </ul>

In [8]:
mean_absolute_error(y, y_predictions)

59.62431715362864

<h1>Generalizing to Unseen Data</h1>
<ul>
    <li>The error on the <b>training set</b> is called the <b>training error</b>
        (also 'resubstitution error' and 'in-sample error').
    </li>
    <li>But we want to know how well we will perform in the future, on <em>unseen data</em>.
        <ul>
            <li>The training error is not, in general a good indicator of performance on unseen data.</li>
            <li>It's often too optimistic. Why?</li>
        </ul>
    </li>
    <li>To predict future performance, we need to measure error on an <em>independent</em> dataset:
        <ul>
            <li>We want a dataset that has played no part in creating the model.</li>
            <li>This second dataset is called the <b>test set</b>.</li>
            <li>The error on the test set is called the <b>test error</b> (also 'out-of-sample error' and
                'extra-sample error').
            </li>
       </ul>
   </li>
</ul>

<h1>Holdout</h1>
<ul>
    <li>So we use the following method:
       <ul>
           <li><em>Shuffle</em> the dataset and <em>partition</em> it into two:
               <ul>
                   <li>training set (e.g. 80% of the full dataset); and</li>
                   <li>test set (the rest of the full dataset).</li>
                </ul>
           </li>
           <li>Train the estimator on the training set.</li>
           <li>Test the model (evaluate the predictions) on the test set.</li>
       </ul>
   </li>
   <li>
       This method is called the <b>holdout</b> method, because the test set
       is withheld (held-out) during training.
       <ul>
           <li>It is essential that the test set is not used in any way to create
               the model.
           </li>
           <li><em>Don't even look at it!</em>
           </li>
           <li>'Cheating' is called <b>leakage</b>.</li>
           <li>('Cheating' is one cause of <b>overfitting</b>; see later.)</li> <!-- (see <i>CS4619</i>)</li>-->
       </ul>
    </li>
    <li>Class exercise: Standardization, as we know, is about scaling the data. It requires calculation
        of the mean and standard deviation. When should the mean and standard deviation be calculated:
        (a) before splitting, on the entire dataset, or (b) after splitting, on just the training set?
        Why?
    </li>
</ul>

<h2>Holdout in scikit-learn: one method</h2>

In [9]:
# Shufle and split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [10]:
# Fit the model on the training data
linear_model.fit(X_train, y_train)

LinearRegression()

In [11]:
# Predict on the test data
y_test_predictions = linear_model.predict(X_test)

# Calculate MAE on the test data
mean_absolute_error(y_test, y_test_predictions)

64.44261488699179

<h2>Holdout in scikit-learn: another way</h2>
<ul>
    <li>This alternative involves writing less code.</li>
</ul>

In [21]:
# Create the object that shuffles and splits the data
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=2)

In [22]:
# Shuffle & split the data, fit the model on the training data, predict on the test data, calculate MAE on the test data
cross_val_score(linear_model, X, y, scoring="neg_mean_absolute_error", cv=ss)

array([-64.44261489])

<ul>
    <li>This is the negative of the MAE &mdash; so that higher values (closer to zero ) are better.</li>
    <li>Often this number (the test error) will be higher than the training error.</li>
    <li>Run it again with a different random state.</li>
</ul>

<h1>Pros and Cons of Holdout</h1>
<ul>
    <li>The advantage of holdout is:
        <ul>
            <li>The test error is independent of the training set.</li>
        </ul>
    </li>
    <li>The disadvantages of this method are:
        <ul>
            <li>Results can vary quite a lot across different runs.
                <ul>
                    <li>Informally, you might get lucky &mdash; or unlucky</li>
                </ul>
                I.e. in any one split, the data used for training or testing might not be representative.
            </li>
            <li>We are training on only a subset of the available dataset, perhaps as little as 50% of it.
                <ul>
                    <li>From so little data, we may learn a worse model and so our error measurement may 
                        be pessimistic.
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li>In practice, we only use the holdout method when we have a very large dataset.
        <ul>
            <li>The size  of the dataset mitigates the above problems.</li>
        </ul>
    </li>
    <li>
        When we have a smaller dataset, we use a <b>resampling</b> method:
        <ul>
            <li>The examples get re-used for training and testing.</li>
        </ul>
    </li> 
</ul>

<h1>$k$-Fold Cross-Validation</h1>
<ul>
    <li>The most-used resampling method is $k$-fold cross-validation:
        <ul>
            <li>Shuffle the dataset and partition it into $k$ disjoint subsets of equal size.
                <ul>
                    <li>Each of the partitions is called a <b>fold</b>.</li>
                    <li>Typically, $k = 10$, so you have 10 folds.</li>
                    <!--
                    <li>But, for conventional statistical significance testing to be applicable, you should probably ensure 
                        that the number of examples in each fold does not fall below 30. (If this isn't possible, then either 
                        use a smaller value for $k$, or do not use $k$-fold cross validation!)
                    </li>
                    -->
                </ul>
            </li>
            <li>You take each fold in turn and use it as the test set, training the learner on 
                the remaining folds.
            </li>
            <li>Clearly, you can do this $k$ times, so that each fold gets 'a turn' at being the test set.
                <ul>
                    <li>
                        By this method, each example is used exactly once for testing, and $k - 1$ times for training.
                    </li>
                </ul>
            </li>
        </ul>
    <li>In pseudocode:
        <ul style="background: lightgray; list-style: none">
            <li>
                shuffle the dataset $D$ and partition it into $k$ disjoint equal-sized subsets, $T_1, T_2,\ldots,T_k$
            <li>
            <li>
                <b>for</b> $i = 1$ to $k$
                <ul>
                    <li>train on $D \setminus T_i$</li>
                    <li>make predictions for $T_i$</li>
                    <li>measure error (e.g. MAE)</li>
                </ul>
                report the mean of the errors
            </li>
        </ul>
    </li>
</ul>

<h2>$k$-Fold Cross Validation in scikit-learn</h2>

In [28]:
# Create the object that shuffles & splits the data
kf = KFold(n_splits=10, shuffle=True, random_state=2)

# Shuffle & split the data
# Repeat k times: fit the model on all but one fold, predict on the remaining fold, calculate MAE
# This gives k MAEs, so take their mean
np.mean(cross_val_score(linear_model, X, y, scoring="neg_mean_absolute_error", cv=kf))

-60.79942430798836

<ul>
    <li>But $k$-fold cross-validation is so common, there's a shorthand:</li>
</ul>   

In [24]:
np.mean(cross_val_score(linear_model, X, y, scoring="neg_mean_absolute_error", cv=10))

-60.20143514563968

<ul>
    <li>Be warned, however, the shorthand does not shuffle the dataset before splitting it into folds.
        <ul>
            <li>Why might that be a problem?</li>
        </ul>
    </li>
    <li>If you use the shorthand, you should probably shuffle the <code>DataFrame</code> just after reading it in from 
        the CSV file (see example below).
    </li>
</ul>

<h2>Pros and Cons of $k$-Fold Cross-Validation</h2>
<ul>
    <li>Pros:
        <ul>
            <li>
                The test errors of the folds are independent &mdash; because examples are included in only one test set.
            </li>
            <li>
                Better use is made of the dataset: for $k = 10$, for example, we train using 9/10 of the dataset.
            </li>
        </ul>
    </li>
    <li>Cons:
        <ul>
            <li>
                While the test sets are independent of each other, the training sets are not: 
                <ul>
                    <li>They will overlap with each other to some degree.</li>
                    <li>(This effect of this will be less, of course, for larger datasets.)</li>
                </ul>
            </li>
            <li>
                The number of folds is constrained by the size of the dataset and the desire sometimes on the part of
                statisticians to have folds of at least 30 examples.
            </li>
            <li>
                It can be costly to train the learning algorithm $k$ times.
            </li>
            <li>
                There may still be some variability in the results due to 'lucky'/'unlucky' splits.
            </li>
        </ul>
    </li>
</ul>

<h1>A Little Case Study</h1>
<ul>
    <li>Let's learn a linear model and compare it with 3NN.</li>
    <li>For 3NN, we will need to standardize the data.</li>
    <li>We will also standardize it for the linear model. scikit-learn's <code>LinearRegression</code> class
        does not require us to do this but no harm is done by doing it. The advantage is that it makes our
        code for the two regressors more consistent.
    </li>
    <li>We'll use 10-fold cross-validation and we'll use the shorthand so we'll shuffle the dataset ourselves.</li>
</ul>

In [29]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("../datasets/dataset_corkA.csv")

# Shuffle the dataset
df = df.sample(frac=1, random_state=2)
df.reset_index(drop=True, inplace=True)

# The features we want to select
features = ["flarea", "bdrms", "bthrms"]

# Extract the features but leave as a DataFrame
X = df[features]

# Target values, converted to a 1D numpy array
y = df["price"].values

In [30]:
# Create a preprocessor
preprocessor = ColumnTransformer([
        ("scaler", StandardScaler(), features)], 
        remainder="passthrough")

In [31]:
# Create a pipeline that combines the preprocessor with the linear model
linear_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", LinearRegression())])

In [32]:
# Create a pipeline that combines the preprocessor with 3NN
knn_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", KNeighborsRegressor(n_neighbors=3))])

In [33]:
# Error estimation for the linear model
np.mean(cross_val_score(linear_model, X, y, scoring="neg_mean_absolute_error", cv=10))

-60.79942430798835

In [34]:
# Error estimation for 3NN
np.mean(cross_val_score(knn_model, X, y, scoring="neg_mean_absolute_error", cv=10))

-72.60975948196116

 <ul>
    <li>Notice how much work <code>cross_val_score</code> is doing for us:
        <ul>
            <li>It partitions the data.</li>
            <li>Then, on the training set, for each transformer in the pipeline, it calls <code>fit</code>
                and <code>transform</code> and, for the predictor at the end of the pipeline, it also
                calls <code>fit</code>.
                <img src="images/pipeline1.png" />
            </li>
            <li>Then, on the test set, for each transformer in the pipeline, it calls <code>transform</code> and, 
                for the predictor at the end of the pipeline, it calls <code>predict</code>.
                <img src="images/pipeline2.png" />
            </li>
            <li>And, in the case of $k$-fold cross-validation, it repeats the above $k$ times.
        </ul>
    </li>
</ul>

<h2>Some remarks</h2>
<ul>
    <li>
        In the past, students have tried holdout and $k$-fold as if they were in
        competition with each other. This betrays a misunderstanding. You do not try them
        both and see which one gives the lower error. You pick one of them &mdash; the one
        that makes most sense for your data &mdash; and use it.
    </li>
    <li>
        There are many resampling methods other than $k$-Fold Cross-Validation:
        <ul>
            <li>Repeated $k$-Fold Cross-Validation, Leave-One-Out-Cross-Validation, 
                &hellip;
            </li>
            <li>See the classes in <code>sklearn.model_selection</code>.</li>
        </ul>
    </li> 
    <li>
        So you've used one of the above methods and found the test error of your predictor.
        <ul>
            <li>This is supposed to give you an idea of how your predictor will perform in practice.</li>
            <li>What if you are dissatisfied with the test error? It seems too high.
                <ul>
                    <li>It is tempting to tweak your learning algorithm or try different algorithms
                        to try to bring down the test error.
                    </li>
                    <li>This is wrong! It is <b>leakage</b> again: you will be using knowledge of the test 
                        set to develop the predictor and is likely to result in an optimistic view of the 
                        ultimate performance of the predictor on unseen data.
                    </li>
                    <li>Ideally, error estimation on the test set is the last thing you do.
                </ul>
            </li>
        </ul>
    </li>
    <li>
        Finally, suppose you have used one of the above methods to estimate the error of your predictor. 
        You are ready to release your predictor on the world. At this point, you can train it on
        <em>all</em> the examples in your dataset, so as to maximize the use of the data.
    </li>
</ul>

<h2>Finishing the case study</h2>
<ul>
    <li>Since the linear model is better than 3NN, this is the model we will deploy.</li>
    <li>At this point, we can retrain on the entire dataset.</li>
</ul>

In [35]:
# Train the linear model on the entire dataset
linear_model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['flarea', 'bdrms',
                                                   'bthrms'])])),
                ('predictor', LinearRegression())])

<ul>
    <li>We can save this model using <code>dump</code> from <code>pickle</code> or <code>joblib</code>.</li>
</ul>

In [36]:
dump(linear_model, "models/my_model.pkl") # For this to work, create a folder called models!

['models/my_model.pkl']

<ul>
    <li>We can read it into, e.g., our web app's backend using <code>load</code>.</li>
</ul>

In [37]:
model = load("models/my_model.pkl")

<ul>
    <li>Then, when we want to make predictions, we can create a <code>DataFrame</code> of objects for which 
        we want predictions and call <code>model.predict</code>.
    </li>
</ul>